In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import time

In [2]:
POSTGRES_USERNAME = "postgresAdmin"
POSTGRES_PASSWORD = "Aloha123"
POSTGRES_DB       = "newsdb"
POSTGRES_HOST     = "rds-postgresql-news.c5pvcyphf6up.ap-southeast-2.rds.amazonaws.com"
POSTGRES_PORT     = '5432'

In [3]:
alchemy_engine = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
pg_engine = f"user='{POSTGRES_USERNAME}' password='{POSTGRES_PASSWORD}' host='{POSTGRES_HOST}' dbname='{POSTGRES_DB}'"

In [4]:
engine = create_engine(alchemy_engine)

In [7]:
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import composite, sessionmaker

In [9]:
import pytz
from datetime import datetime

class Article:
    def __init__(self, article_id, href, title, date, newspaper, content, author_id=None, author_fullname = '', tags=[]):
        self._id = article_id
        self._href = href
        self._title = title.strip()
        self._date = date  # date is stored in UTC timezone
        self._newspaper = newspaper
        self._creation_date = pytz.utc.localize(datetime.utcnow()) # set utc now time
        self._author_id = author_id
        self._author_fullname = author_fullname
        self._content = content # content is a list of dict [{'type':'text', 'content':''}, {'type':'image', 'link':'url'}]
        self._tags = tags
    
    def get_id(self):
        return self._id
    
    def get_href(self):
        return self._href
    
    def get_title(self):
        return self._title
    
    def get_date(self):
        if type(self._date) != bool:
            return self._date
        else:
            return pytz.utc.localize(datetime.utcnow())
        
    def get_newspaper(self):
        return self._newspaper
        
    def get_author_id(self):
        return self._author_id
    
    def get_author_fullname(self):
        return self._author_fullname
    
    def get_content(self):
        return self._content
    
    def get_tags(self):
        return self._tags

#     def get_category(self):
#         return self._category


In [18]:
Base = declarative_base()

class Postgres_Article(Base):
    __tablename__ = 'articles'
    # basic fields
    article_id = Column(String, primary_key=True, nullable=False)
    title = Column(String, nullable=False)
    href = Column(String, nullable=False)
    publish_date = Column(Date)
    newspaper = Column(String)
    author_id = Column(String)
    author_fullname = Column(String)
    content = Column(String, nullable=False) # full content

    # analytic fields
    sentimental = Column(String)

class PostgresClient():
    def __init__(self):
        db_string = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
        db = create_engine(db_string)
        Session = sessionmaker(db)
        self._session = Session()
        Base.metadata.create_all(db)
        
    def push_article(self, article):
        new_article = Postgres_Article(
            article_id = article.get_id(),
            title = article.get_title(),
            href  = article.get_href(),
            publish_date = article.get_date(),
            newspaper = article.get_newspaper(),
            author_id = article.get_author_id(),
            author_fullname = article.get_author_fullname(),
            content = article.get_content()
        )
        
        self._session.add(new_article)
        self._session.commit()
        
    def get_sample_articles(self):
        for article in self._session.query(Postgres_Article):
            print(f"Title: {article.title}")
            print(f"Newpspaper: {article.newspaper}")
            print(f"Link: {article.href}")
            print(f"Publish date: {article.publish_date}")
            print(f"Author: {article.author_fullname}")
            print(f"Content: {article.content}")
            print()

            input("Press Enter to print next article !")
            print()
            
    def get_session(self):
        return self._session
            
# sample_article = Article(
#     article_id = np.random.randint(0, 1000000),
#     href = 'https://test.com',
#     title = 'day la bai test',
#     date = None,
#     newspaper = 'CNN',
#     author_id = '100',
#     author_fullname = 'Ha Nguyen',
#     content = """
#     asdf
#     asdf
#     asdf
#     ffdfksdmkfmskd
#     """
# )
# print("Test push a sample article")
db = PostgresClient()
# db.push_article(sample_article)
# print("OK")

print("Test list articles in database")
db.get_sample_articles()
print("OK")

Test list articles in database
Title: day la bai test 2
Newpspaper: CNN
Link: https://test.com
Publish date: 2022-07-16
Author: Ha Nguyen
Content: 
    xcvxsdfsdfsfsdfsdf
    
    sdfsdf
    sdf
    

Press Enter to print next article !

Title: Sri Lanka crisis: How do you fix a broken country?
Newpspaper: CNN
Link: /2022/07/15/asia/sri-lanka-crisis-economy-fix-gotabaya-rajapaksa-saturday-intl-hnk/index.html
Publish date: 2022-07-16
Author: By Heather Chen, CNN
Content: But all that changed on July 9, when protesters stormed in and took control, demanding the resignation of President Gotabaya Rajapaksa before turning the palace upside down.
"That was the home of the most powerful man in the country," said Sri Lankan author and analyst Asanga Abeyagoonasekera. "It had never been opened to the public." 
It's now become a novelty attraction -- all traces of its exclusivity and prestige gone. Each day for the past five days, thousands have lined-up up for hours just for a glimpse of Rajapa

Press Enter to print next article !

Title: Sri Lanka crisis: How do you fix a broken country?
Newpspaper: CNN
Link: /2022/07/15/asia/sri-lanka-crisis-economy-fix-gotabaya-rajapaksa-saturday-intl-hnk/index.html
Publish date: 2022-07-16
Author: By Heather Chen, CNN
Content: But all that changed on July 9, when protesters stormed in and took control, demanding the resignation of President Gotabaya Rajapaksa before turning the palace upside down.
"That was the home of the most powerful man in the country," said Sri Lankan author and analyst Asanga Abeyagoonasekera. "It had never been opened to the public." 
It's now become a novelty attraction -- all traces of its exclusivity and prestige gone. Each day for the past five days, thousands have lined-up up for hours just for a glimpse of Rajapaksa's luxurious lifestyle. The neatly manicured lawns have become picnic spots and protesters swim and party in his private pool.

Rajapaksa fled the crisis-hit country on Wednesday, boarding a militar

Press Enter to print next article !

Title: Sri Lanka's former President has stepped down -- but larger problems loom for the crisis-hit country
Newpspaper: CNN
Link: /2022/07/15/asia/sri-lanka-rajapaksa-crisis-friday-intl-hnk/index.html
Publish date: 2022-07-15
Author: By Jessie Yeung, Iqbal Athas and Kunal Sehgal, CNN
Content: The President's departure from office marks a major victory for the protesters, who for months have demanded the removal of both Rajapaksa and Prime Minister Ranil Wickremesinghe. 
Many in Sri Lanka blame Rajapaksa for the country's worsening situation, with runaway inflation and shortages of basic goods such as fuel and food impacting everyday life.
But while Rajapaksa is now out of the picture, having landed in Singapore on Thursday, following an earlier escape to the Maldives via military jet, his close political ally Wickremesinghe remains firmly in place -- and was sworn in as Acting President Friday. 

A senior government source told CNN that Rajapaksa ap

Press Enter to print next article !

Title: Former German soldier gets five-and-a-half years in prison for far-right plot
Newpspaper: CNN
Link: /2022/07/15/europe/german-soldier-franco-a-sentenced-intl-grm/index.html
Publish date: 2022-07-15
Author: By Nadine Schmidt, CNN
Content: The 33-year-old former army officer -- identified as Franco A. in accordance with German privacy laws -- was charged with posing under a false identity as an asylum seeker in 2017 and planning an attack that he apparently hoped would be blamed on refugees and migrants.
''The accused is guilty of planning a serious act of violence, endangering the state,'' presiding judge Christoph Koller said when reading out the verdict at the regional superior court in the western city of Frankfurt.
In the trial that began in May 2021, prosecutors said the Bundeswehr soldier also stole ammunition from the German military, with former Justice Minister Heiko Maas or the parliament's former vice-president, Claudia Roth, seen a

Press Enter to print next article !

Title: Ukraine's harvest becomes the new battlefield, as fires blacken its arable heartlands
Newpspaper: CNN
Link: /2022/07/11/europe/ukraine-harvest-battlefield-intl/index.html
Publish date: 2022-07-16
Author: By Tim Lister and Petro Zadorozhnyy, CNN
Content: The conflict's front lines straddle some of Ukraine's richest farmland. Whether caused by accident or intention, the fires darkening the summer sky are eating into a harvest that was always going to be tough to collect and even tougher to export.  
Pavlo Serhienko is in the crosshairs of this battle. The 24-year-old is the third generation of his family to run a farm in the Vasylivka district of Zaporizhzhia. Since his father died from coronavirus, Serhienko is managing the 3,000-hectare farm on his own.  
But nearly half the land is now too dangerous to cultivate, he told CNN on Saturday.
"We can't even get there. It is either mined or near the occupied territories, literally the front line. 

Press Enter to print next article !

Title: Former German soldier gets five-and-a-half years in prison for far-right plot
Newpspaper: CNN
Link: /2022/07/15/europe/german-soldier-franco-a-sentenced-intl-grm/index.html
Publish date: 2022-07-15
Author: By Nadine Schmidt, CNN
Content: The 33-year-old former army officer -- identified as Franco A. in accordance with German privacy laws -- was charged with posing under a false identity as an asylum seeker in 2017 and planning an attack that he apparently hoped would be blamed on refugees and migrants.
''The accused is guilty of planning a serious act of violence, endangering the state,'' presiding judge Christoph Koller said when reading out the verdict at the regional superior court in the western city of Frankfurt.
In the trial that began in May 2021, prosecutors said the Bundeswehr soldier also stole ammunition from the German military, with former Justice Minister Heiko Maas or the parliament's former vice-president, Claudia Roth, seen a

In [28]:
db = PostgresClient()

In [63]:
Base.metadata.drop_all(bind=engine, tables=[Postgres_Article.__table__])

In [66]:
#sql cmd
#create table
sql_query =f"""SELECT * FROM articles;"""

In [67]:
df = pd.read_sql_query(sql_query, engine)

In [68]:
df

,article_id,title,href,publish_date,newspaper,category,author_id,author_fullname,content,related_articles,sentimental,keywords
